In [1]:
using PyCall
using Distances
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using Distances
using DataFrames
# using PyPlot
using SparseArrays
using Printf
using Images

┌ Info: Recompiling stale cache file C:\Users\user\.juliapro\JuliaPro_v1.1.1.1\compiled\v1.1\PyCall\GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file C:\Users\user\.juliapro\JuliaPro_v1.1.1.1\compiled\v1.1\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file C:\Users\user\.juliapro\JuliaPro_v1.1.1.1\compiled\v1.1\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file C:\Users\user\.juliapro\JuliaPro_v1.1.1.1\compiled\v1.1\Images\H8Vxc.ji for Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1184


In [23]:
py"""
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import sys
import cv2
import os

class SceneReconstruction3D:

    def __init__(self,K, dist, target_width):
        self.K = K
        self.K_inv = np.linalg.inv(K)
        self.d = dist
        self.target_width = target_width

    def load_image_pair(self, img_path1, img_path2, use_pyr_down=True):
        self.img1 = cv2.imread(img_path1, cv2.CV_8UC3)
        self.img2 = cv2.imread(img_path2, cv2.CV_8UC3)

        if self.img1 is None:
            sys.exit("Image " + img_path1 + " could not be loaded.")
        if self.img2 is None:
            sys.exit("Image " + img_path2 + " could not be loaded.")

        if len(self.img1.shape) == 2:
            self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_GRAY2BGR)
            self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_GRAY2BGR)

        
        if use_pyr_down and self.img1.shape[1] > self.target_width:
            while self.img1.shape[1] > 2 * self.target_width:
                self.img1 = cv2.pyrDown(self.img1)
                self.img2 = cv2.pyrDown(self.img2)

        self.img1 = cv2.undistort(self.img1, self.K, self.d)
        self.img2 = cv2.undistort(self.img2, self.K, self.d)

    def findRootSIFTFeatures(self, n_components):
        class RootSIFT:
            def __init__(self):
                self.extractor = cv2.xfeatures2d.SIFT_create(n_components)

            def compute(self, image, kps, eps=1e-7):
                (kps, descs) = self.extractor.compute(image, kps)
                if len(kps) == 0:
                    return ([], None)

                descs /= (descs.sum(axis=1, keepdims=True) + eps)
                descs = np.sqrt(descs)
                return (kps, descs)

        class InnerFeatures:
            def __init__(self, kps, des, pos):
                self.kps = kps
                self.des = des
                self.pos = pos

        def innerRootSIFT(img):
            sift = cv2.xfeatures2d.SIFT_create(n_components)
            (kps, descs) = sift.detectAndCompute(img, None)

            rs = RootSIFT()
            (kps, descs) = rs.compute(img, kps)
            pos = np.float32([np.array([x.pt[0], x.pt[1]]) for x in kps])

            # cleaning
            return kps, descs, pos

        kps1, desc1, pos1 = innerRootSIFT(self.img1)
        kps2, desc2, pos2 = innerRootSIFT(self.img2)
        self.feature_1 = InnerFeatures(kps1, desc1, pos1)
        self.feature_2 = InnerFeatures(kps2, desc2, pos2)

    def drawMatches(self, path):
        self.outImage = cv2.drawMatches(self.img1, self.feature_1.kps, self.img2, self.feature_2.kps, self.matches,outImg=None)
        cv2.imwrite(path, self.outImage)
    
    def _find_fundamental_matrix(self):
        self.F, self.Fmask = cv2.findFundamentalMat(self.match_pts1,
                                                    self.match_pts2,
                                                    cv2.FM_RANSAC, 0.1, 0.99)

    def _find_essential_matrix(self):
        self.E = self.K.T.dot(self.F).dot(self.K)

    def _find_camera_matrices_rt(self):
        # decompose essential matrix into R, t (See Hartley and Zisserman 9.13)
        U, S, Vt = np.linalg.svd(self.E)
        W = np.array([0.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
                      1.0]).reshape(3, 3)

        # iterate over all point correspondences used in the estimation of the
        # fundamental matrix
        first_inliers = []
        second_inliers = []
        for i in range(len(self.Fmask)):
            if self.Fmask[i]:
                # normalize and homogenize the image coordinates
                first_inliers.append(self.K_inv.dot([self.match_pts1[i][0],
                                                     self.match_pts1[i][1], 1.0]))
                second_inliers.append(self.K_inv.dot([self.match_pts2[i][0],
                                                      self.match_pts2[i][1], 1.0]))

        # Determine the correct choice of second camera matrix
        # only in one of the four configurations will all the points be in
        # front of both cameras
        # First choice: R = U * Wt * Vt, T = +u_3 (See Hartley Zisserman 9.19)
        R = U.dot(W).dot(Vt)
        T = U[:, 2]
        if not self._in_front_of_both_cameras(first_inliers, second_inliers,
                                              R, T):
            # Second choice: R = U * W * Vt, T = -u_3
            T = - U[:, 2]

        if not self._in_front_of_both_cameras(first_inliers, second_inliers,
                                              R, T):
            # Third choice: R = U * Wt * Vt, T = u_3
            R = U.dot(W.T).dot(Vt)
            T = U[:, 2]

            if not self._in_front_of_both_cameras(first_inliers,
                                                  second_inliers, R, T):
                # Fourth choice: R = U * Wt * Vt, T = -u_3
                T = - U[:, 2]

        self.match_inliers1 = first_inliers
        self.match_inliers2 = second_inliers
        self.Rt1 = np.hstack((np.eye(3), np.zeros((3, 1))))
        self.Rt2 = np.hstack((R, T.reshape(3, 1)))

    def _in_front_of_both_cameras(self, first_points, second_points, rot,trans):
        rot_inv = rot
        for first, second in zip(first_points, second_points):
            first_z = np.dot(rot[0, :] - second[0] * rot[2, :],trans) / np.dot(rot[0, :] - second[0] * rot[2, :],second)
            first_3d_point = np.array([first[0] * first_z,second[0] * first_z, first_z])
            second_3d_point = np.dot(rot.T, first_3d_point) - np.dot(rot.T,trans)

            if first_3d_point[2] < 0 or second_3d_point[2] < 0:
                return False

        return True
K = np.array([[2759.48/4, 0, 1520.69/4, 0, 2764.16/4,1006.81/4, 0, 0, 1]]).reshape(3, 3)
d = np.array([0.0, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5)
scene = SceneReconstruction3D(K,d,target_width=800)

"""
img1_path = "../data/pair/Left.png"
img2_path = "../data/pair/Right.png"

py"scene.load_image_pair"(img1_path, img2_path)
py"scene.findRootSIFTFeatures"(1300)

pts1 = py"scene.feature_1.pos"
pts2 = py"scene.feature_2.pos";

P_points = pts1
Q_points = pts2

println("size P points", size(P_points))
println("size Q points", size(Q_points))

size P points(1300, 2)
size Q points(1300, 2)


In [24]:
cost = pairwise(Euclidean(), P_points, Q_points; dims=1)
println(size(cost))
P = ones(size(P_points,1))
Q = ones(size(Q_points,1));

(1300, 1300)


In [25]:
solCount = 10
# m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0,PoolGap = 0.001))
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(P), sum(Q)))
@constraint(m, X * ones(Int, length(Q)) .<= P)
@constraint(m, X'ones(Int, length(P)) .<= Q);
optimize!(m)
solution_pool = zeros(solCount, length(P),length(Q))
cnt = 0
obj = objective_value(m)

for i in 0:(solCount-1)
    try
        setparam!(m.moi_backend.inner,"SolutionNumber", i)
        xn = Gurobi.get_dblattrarray(m.moi_backend.inner, "Xn", 1, length(X))
        xn_val = Gurobi.get_dblattr(m.moi_backend.inner, "PoolObjVal")
        if(round(xn_val,digits=1) != round(obj, digits=1))
            println(cnt , " solution(s) selected")
            break
        end
        default = zeros(length(P),length(Q))
        for i in 0:length(P)-1
            default[i+1,:] = xn[(i*length(Q))+1:(i+1)*length(Q)]
        end
        solution_pool[i+1,:,:] = default
        cnt+=1
    catch 
        break
    end
end
sol_pool = deepcopy(solution_pool[1:cnt,:,:]);

Optimize a model with 2601 rows, 1690000 columns and 5070000 nonzeros
Variable types: 0 continuous, 1690000 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve time: 6.63s
Presolved: 2601 rows, 1690000 columns, 5070000 nonzeros
Variable types: 0 continuous, 1690000 integer (1690000 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.300000e+03   0.000000e+00     15s

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00     15s
       1       3889   2.2502494e+08   1.4923464e+04     15s
       2      11197   1.5403447e+08   2.4606230e+04     16s
       3      19096   1.2904006e+08   3.2349578e+04     16s


1 solution(s) selected


In [26]:
solOther = sparse(sol_pool[1,:,:])
sizeOf = min(size(P,1), size(Q,1))
matched_pts1 = zeros(sizeOf,2)
matched_pts2 = zeros(sizeOf,2)
i = 1
py"""
arr = []
"""
for (x,y,v) in zip(findnz(solOther)...)
    x_pos = [P_points'[:,x][1], Q_points'[:,y][1]]
    y_pos = [P_points'[:,x][2], Q_points'[:,y][2]]
    
    # dmatch creating
    queryId = x-1
    trainId = y-1
    distance = cost[x,y]
    if(distance <= 10)
        dmatch = py"cv2.DMatch($(queryId), $(trainId),$(distance))"
        py"arr.append"(dmatch)
        matched_pts1[i,:] = [floor(x_pos[1]) floor(y_pos[1])]
        matched_pts2[i,:] = [floor(x_pos[2]) floor(y_pos[2])]
        i+=1
    end
end
py"""
scene.matches = arr
"""
path = "../data\\pair\\lastLPMatched.png"
py"scene.drawMatches"(path)


matched_final_1 = deepcopy(matched_pts1[1:i-1, :])
matched_final_2 = deepcopy(matched_pts2[1:i-1, :]);

In [27]:
df = DataFrame()
df.PX = matched_final_1[:,1]
df.PY = matched_final_1[:,2]
df.QX = matched_final_2[:,1]
df.QY = matched_final_2[:,2];
df

,PX,PY,QX,QY
,Float64,Float64,Float64,Float64
1,505.0,70.0,509.0,71.0
2,499.0,54.0,499.0,50.0
3,1424.0,140.0,1427.0,140.0
4,670.0,409.0,675.0,403.0
5,667.0,34.0,674.0,37.0
6,670.0,284.0,673.0,275.0
7,665.0,508.0,672.0,503.0
8,679.0,343.0,670.0,341.0
9,662.0,497.0,666.0,497.0


In [28]:
CSV.write("../data/pair/matchedPoints.csv",  df, writeheader=false)

"../data/pair/matchedPoints.csv"